In [17]:
from PyQt5 import QtWidgets
from PyQt5.QtWidgets import * 
import sys
import pandas as pd
import numpy as np
import random

In [ ]:
# def dialog():
#     mbox = QMessageBox()
    
#     mbox.setGeometry(300, 150, 1000, 800)

#     mbox.setText("Print user input here")
#     mbox.setStandardButtons(QMessageBox.Ok | QMessageBox.Cancel)
            
#     mbox.exec_()

In [18]:
df_recipes = pd.read_csv("../datasets/Recipes.csv")
df_ing = pd.read_csv("../datasets/Ingredients.csv")
df_rec2ing = pd.read_csv("../datasets/Rec2Ing.csv")

In [ ]:
#Define all the functions here

#Searches through recipe title and ingredient title to find ones that CONTAINS the allergen
#Then filters the df_recipes dataframe to remove the allergens containing recipes
#Doing it this way to reduce time taken on merging datasets
#Searches based on df.str.contains function 
#Applies it directly to the df_recipes dataframe

def allergy_filter(allergies):

    if len(allergies)>=1 and allergies[0]!='':

        alg = [i+'*' for i in allergies]

        #get all ingredients whose name matches the allergies; merge it with rec2ing to get a list of recipeIDs
        dfi = df_ing[df_ing.IngredientName.str.lower().str.contains('|'.join(alg), na=False, regex=True)]
        rec2ingID_list = list(df_rec2ing.merge(dfi, on='IngredientID')['RecipeID'])

        #get all recipes whose title matches the allergies
        recList = list(df_recipes[df_recipes.Title.str.lower().str.contains('|'.join(alg), na=False, regex=True)]['RecipeID'])
        recList.append(rec2ingID_list)
        dfr = df_recipes[~df_recipes.RecipeID.isin(recList)]
    else:
        dfr = df_recipes
    
    return dfr


#Filter the inputted df using the isVegan attribute based on user input

def vegan_filter(vegan, dfr):

    if vegan == 'Vegan':
        df = dfr[dfr.isVegan=='1']
    elif vegan == "Strictly not vegan":
        df = dfr[dfr.isVegan=='0']
    else:
        df = dfr
        
    return df


#Exactly the same as allergy_filter. Only this time we return the df elemtns containing the search_term

def search_filter(searc_term, df):

    if len(search_term)>=1 and search_term[0]!='':

        srch = [i+'*' for i in search_term]

        #get all ingredients whose name matches the allergies; merge it with rec2ing to get a list of recipeIDs
        dfi = df_ing[df_ing.IngredientName.str.lower().str.contains('|'.join(srch), na=False, regex=True)]
        rec2ingID_list = list(df_rec2ing.merge(dfi, on='IngredientID')['RecipeID'])

        #get all recipes whose title matches the allergies
        recList = list(df[df.Title.str.lower().str.contains('|'.join(srch), na=False, regex=True)]['RecipeID'])
        recList.append(rec2ingID_list)
        dfr = df[df.RecipeID.isin(recList)]
    else:
        dfr = df
    
    return dfr


#Filters based on calculated nutritional facts adding a +/- 10% buffer

def cal_filter(cal, protein, fat, df):
    return df[(df.Calories >= 0.9*cal) & (df.Calories <= 1.1*cal) & (df.Protein >= 0.9*protein) & (df.Protein >= 0.9*protein) 
              & (df.Fats >= 0.9*fat) & (df.Fats >= 0.9*fat)]


#Incorporates the above functions to return a list of recipe IDs for individual recipe search

def ind_recipes(x, cals, protein, fat, search_term, vegan, allergies):
    
    df = cal_filter(cals, protein, fat, search_filter(search_term, vegan_filter(vegan, allergy_filter(allergies))))
    
    df = df[df.No_of_reviews >= 10]
    df['score'] = (df.Average_rating*np.log10(df.No_of_reviews)) - (x*df.Prep_time/60)
    df.sort_values('score', ascending=False, inplace=True)
    
    ind_list = list(df['RecipeID'].iloc[0:14])
    filtered_list = random.sample(ind_list, k = 5)
    
    return filtered_list


#Uses the above functions (not ind_recipes) to return a list of lists
#Each child list corresponds to the meals (so 4 child lists for 4 meals/day)
#Each element in the child list corresponds to the recipe options user can select from (3/5 options per child list - defined by k)

def meal_plan(x, cals, protein, fat, vegan, allergies, meals):
    
    df = cal_filter(cals, protein, fat, vegan_filter(vegan, allergy_filter(allergies)))
    
    df = df[df.No_of_reviews >= 10]
    df['score'] = (df.Average_rating*np.log10(df.No_of_reviews)) - (x*df.Prep_time/60)
    df.sort_values('score', ascending=False, inplace=True)
    
    ind_list = list(df['RecipeID'].iloc[0:5*meals])
    filtered_list = [random.sample(ind_list, k = 3) for i in range(0,meals)]
    
    return filtered_list

#hardcoded here, this should be a user input
x=5

In [2]:
class Window(QDialog):
  
    def __init__(self):
        super(Window, self).__init__()
        self.setStyleSheet("background-color: white;")
        self.setWindowTitle("Recipe Recommender and Grocery Shopping")
        self.setGeometry(300, 150, 1000, 800)

        self.form = QGroupBox("Dietary Metrics")
        self.age = QSpinBox()
        self.gender = QComboBox()
        self.gender.addItems(["Male", "Female", "N/A"])
        self.heightF = QComboBox()
        self.heightF.addItems(["0", "1", "2", "3", "4", "5", "6", "7"])
        self.heightI = QComboBox()
        self.heightI.addItems(["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11"])
        self.activity = QComboBox()
        self.activity.addItems(["Little-no exercise", "Light exercise, 1-3 days/week", "Moderate exercise, 3-5 days/week", 
                                "Hard exercise, 6-7 days a week", "Very hard exercise or 2x training"])
        self.weight = QLineEdit()
        self.meals = QLineEdit()
        self.allergies = QLineEdit()
        self.diet = QComboBox()
        self.diet.addItems(["Vegan", "Strictly not vegan", "Don't mind either"])
        self.output = QComboBox()
        self.output.addItems(["Single recipe", "Meal plan"])
        self.pref = QLineEdit()

        self.createForm()

        self.buttonBox = QDialogButtonBox(QDialogButtonBox.Ok | QDialogButtonBox.Cancel)
        self.buttonBox.accepted.connect(self.getInfo)
        self.buttonBox.rejected.connect(self.reject)
        mainLayout = QVBoxLayout()
        mainLayout.addWidget(self.form)
        mainLayout.addWidget(self.buttonBox)
        self.setLayout(mainLayout)

        self.requirements = QtWidgets.QLabel(self)
        self.requirements.move(240, 470)
        self.requirements.resize(105,30)
        self.define = QtWidgets.QLabel(self)
        self.define.move(230, 500)
        self.define.resize(125,30)

        self.D = QtWidgets.QLabel(self)
        self.D.move(195, 530)
        self.D.resize(70,30)
        self.PS = QtWidgets.QLabel(self)
        self.PS.move(190, 560)
        self.PS.resize(75,30)
        self.PM = QtWidgets.QLabel(self)
        self.PM.move(205, 590)
        self.PM.resize(60,30)
        self.PSPM = QtWidgets.QLabel(self)
        self.PSPM.move(130, 620)
        self.PSPM.resize(135,30)

        self.DV = QtWidgets.QLabel(self)
        self.DV.move(270, 530)
        self.DV.resize(160,30)
        self.PSV = QtWidgets.QLabel(self)
        self.PSV.move(270, 560)
        self.PSV.resize(160,30)
        self.PMV = QtWidgets.QLabel(self)
        self.PMV.move(270, 590)
        self.PMV.resize(160,30)
        self.PSPMV = QtWidgets.QLabel(self)
        self.PSPMV.move(270, 620)
        self.PSPMV.resize(160,30)
        

    def getInfo(self):
        print("Age : {0}".format(self.age.text()))
        print("Gender : {0}".format(self.gender.currentText()))
        print("Height(feet) : {0}".format(self.heightF.currentText()))
        print("Height(inches) : {0}".format(self.heightI.currentText()))
        print("Activity : {0}".format(self.activity.currentText()))
        print("Weight : {0}".format(self.weight.text()))
        print("Meals per Day : {0}".format(self.meals.text()))
        print("Allergies : {0}".format(self.allergies.text()))
        print("Diet : {0}".format(self.diet.currentText()))
        print("Preferred Meals/Ingredients : {0}".format(self.pref.text()))
        self.validateInput()
        self.printRequirements()

    def validateInput(self):
        self.all_data = {}

        if self.age.text().isnumeric():
            self.all_data['age'] = int(self.age.text())
        else:
            self.all_data['age'] = 30

        self.all_data['gender'] = self.gender.currentText()

        if self.heightF.currentText().isnumeric():
            self.all_data['height feet'] = int(self.heightF.currentText())
        else:
            self.all_data['height feet'] = 5

        if self.heightI.currentText().isnumeric():
            self.all_data['height inches'] = int(self.heightI.currentText())
        else:
            self.all_data['height inches'] = 6

        self.all_data['activity'] = self.activity.currentText()

        if self.weight.text().isnumeric():
            self.all_data['weight'] = int(self.weight.text())
        else:
            self.all_data['weight'] = 140

        if self.meals.text().isnumeric():
            self.all_data['meals'] = int(self.meals.text())
        else:
            self.all_data['meals'] = 3

        self.all_data['allergies'] = self.allergies.text().split(", ")

        self.all_data['diet'] = self.diet.currentText()

        self.all_data['preferences'] = self.pref.text().split(", ")

    def printRequirements(self):
        self.requirements.setText('REQUIREMENTS:')
        self.define.setText('Calories/Protein/Fat')

        self.D.setText('Total Daily:')
        self.PM.setText('Per Meal:')

        self.caloriesD = self.getCalories()
        self.caloriesPM = self.caloriesD / self.all_data['meals']

        self.proteinD = self.getProtein()
        self.proteinPM = self.proteinD / self.all_data['meals']

        self.fatD = self.getFat()
        self.fatPM = self.fatD / self.all_data['meals']

        self.DV.setText(str(round(self.caloriesD, 2)) + '/' + str(round(self.proteinD, 2)) + 'g' + '/' + str(round(self.fatD, 2)) + 'g')
        self.PMV.setText(str(round(self.caloriesPM, 2)) + '/' + str(round(self.proteinPM, 2)) + 'g' + '/' + str(round(self.fatPM, 2)) + 'g')
        

    def getCalories(self):
        
        self.activity_dict = {"Little-no exercise":1.2,
                            "Light exercise, 1-3 days/week":1.375,
                            "Moderate exercise, 3-5 days/week": 1.55,
                            "Hard exercise, 6-7 days a week": 1.725,
                            "Very hard exercise or 2x training": 1.9}
        
        self.multiplier = self.activity_dict[self.activity.currentText()]
        
        if self.gender.currentText() == "Male":
            return (88.362 + (13.397 * self.all_data['weight'] / 2.205) + (4.799 * (self.all_data['height feet'] * 12 + self.all_data['height inches']) * 2.54) - (5.677 * self.all_data['age'])) * self.multiplier
        elif self.gender.currentText() == "Female":
            return (447.593 + (9.247 * self.all_data['weight'] / 2.205) + (3.098 * (self.all_data['height feet'] * 12 + self.all_data['height inches']) * 2.54) - (4.33 * self.all_data['age'])) * self.multiplier
        else:
            return (267.9775 + (11.322 * self.all_data['weight'] / 2.205) + (3.9485 * (self.all_data['height feet'] * 12 + self.all_data['height inches']) * 2.54) - (5.0035 * self.all_data['age'])) * self.multiplier

    def getProtein(self):
        return self.all_data['weight'] / 20 * 7

    def getFat(self):
        return self.caloriesD * 0.3 / 9   
  
    def createForm(self):
        layout = QFormLayout()
        layout.addRow(QLabel("Age (years)"), self.age)
        layout.addRow(QLabel("Gender"), self.gender)
        layout.addRow(QLabel("Height(feet)"), self.heightF)
        layout.addRow(QLabel("Height(inches)"), self.heightI)
        layout.addRow(QLabel("Activity"), self.activity)
        layout.addRow(QLabel("Weight(lbs)"), self.weight)
        layout.addRow(QLabel("Recommendations for:"), self.output)
        layout.addRow(QLabel("Meals per Day"), self.meals)
        layout.addRow(QLabel("Food Allergies (separate items with a comma)"), self.allergies)
        layout.addRow(QLabel("Vegan?"), self.diet)
        layout.addRow(QLabel("Search keywords (separate items with a comma)"), self.pref)
        self.form.setLayout(layout)

  
if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = Window()
    window.show()
    sys.exit(app.exec())


SystemExit: 0

/Users/hari/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
ingList = ["bacon", "onion", "potatoes", "eggs", "cheddar", "cottage cheese"]
unitList =["pound", "count", "cup", "count", "cup", "cup"]        
amountList = [1, 1, 4, 9, 2, 1.5]

#adjust units for Amazon
for i in range(len(unitList)):
    if unitList[i] in ["cup", "clove", "pound", "ounce", "clove", "stalk", "quart", "slice", "pint","gallon", "drop"]:
        unitList[i] = unitList[i].upper() + "S"
    if unitList[i] == "tablespoon":
        unitList[i] = "TBSP"
    if unitList[i] == "teaspoon":
        unitList[i] = "TSP"
    if unitList[i] == "leaf":
        unitList[i] = "LEAVES"
    if unitList[i] == "pinch":
        unitList[i] = "PINCHES"
    if unitList[i] == "dash":
        unitList[i] = "DE_DASH"

In [19]:
df_ing.shape

(5841, 2)

In [3]:
import json

example = {"ingredients": []}

for ing, un, amt in zip(ingList, unitList,  amountList):
    newD = {"name": ing, 
            "quantityList":[{"unit": un, "amount": amt}],
            "exclusiveOverride": "false"}
    example['ingredients'].append(newD)


# convert into JSON:
myJSON = json.dumps(example)

In [4]:
myJSON

'{"ingredients": [{"name": "bacon", "quantityList": [{"unit": "POUNDS", "amount": 1}], "exclusiveOverride": "false"}, {"name": "onion", "quantityList": [{"unit": "count", "amount": 1}], "exclusiveOverride": "false"}, {"name": "potatoes", "quantityList": [{"unit": "CUPS", "amount": 4}], "exclusiveOverride": "false"}, {"name": "eggs", "quantityList": [{"unit": "count", "amount": 9}], "exclusiveOverride": "false"}, {"name": "cheddar", "quantityList": [{"unit": "CUPS", "amount": 2}], "exclusiveOverride": "false"}, {"name": "cottage cheese", "quantityList": [{"unit": "CUPS", "amount": 1.5}], "exclusiveOverride": "false"}]}'

In [25]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.firefox.options import Options
import time
import webbrowser
import os

In [14]:
options = Options()
options.headless = True
driver = webdriver.Safari(options = options)
driver.get('https://www.amazon.com/afx/ingredients/verify')

#Find the div id of the dropdown menu and select the value needed based on the displayed text 
#Can be done by element value too

#Select(driver.find_element(By.ID,'afx-ingredients-verify-brand-input')).select_by_value("QW1hem9uIEZyZXNo")
#driver.find_element(By.CLASS_NAME,'a-dropdown-container').click()

#select = Select(driver.find_element(By.ID,'afx-ingredients-verify-brand-input'))

#time.sleep(1)
# driver.find_element(By.ID,'afx-ingredients-verify-brand-input').click()
# #(By.XPATH, "//*[@id='afx-ingredients-verify-brand-input']//options[contains(.,'Whole Foods)]")))
# #.select_by_value("VUZHIFdob2xlIEZvb2Rz")
#select.select_by_visible_text("Whole Foods")

#Clear the values in the text box
driver.find_element(By.ID,'afx-ingredients-verifier-json-input').clear()

#Input values using the variable test - right now it uses the test string variable, so it'll stop working after verify as no url would be created
driver.find_element(By.ID,'afx-ingredients-verifier-json-input').send_keys(myJSON)

#Click on the verify button
driver.find_element(By.CLASS_NAME,'a-button-input').click()

#Wait a second for the URl link to show up
time.sleep(1)

#Grab the URL link
href_url = driver.find_element(By.ID,'afx-ingredients-encoded').get_attribute('href')

#Create a new driver that'll open the link on the web browser
driver2 = webdriver.Safari()
driver2.get(href_url)

In [20]:
selectedRecipeIds = [103305, 229150, 11730]
ingList = []
unitList = []
amountList = []

ingId = []
#df_recipes.head()
#df_ing.head()



for rID in selectedRecipeIds:
#rID = selectedRecipeIds[0]

    newDF = df_rec2ing.loc[df_rec2ing['RecipeID'] == rID]
    ingId += newDF['IngredientID'].tolist()
    amountList += newDF['Quantity'].tolist()
    unitList += newDF['Unit'].tolist()

    
print(ingId, amountList, unitList, sep="\n")

for i,iD in enumerate(ingId):
    index = df_ing[df_ing['IngredientID'] == iD].index[0]
    ingList.append(df_ing.iloc[index]['IngredientName'])

# df_ing.head()
print(ingList)

[27, 171, 4, 13, 107, 91, 345, 78, 156, 261, 36, 175, 56, 71, 8512, 20, 3, 14, 36, 80, 4, 21, 105, 103, 127]
[2.0, 2.0, 2.0, 56.0, 30.0, 6.0, 2.0, 1.0, 31.0, 1.0, 1.0, 4.0, 9.0, 2.0, 1.5, 9.0, 1.0, 1.0, 1.0, 1.0, 2.0, 0.5, 2.0, 1.0, 7.5]
['pound', 'cup', 'count', 'ounce', 'ounce', 'teaspoon', 'teaspoon', 'count', 'ounce', 'pound', 'count', 'cup', 'count', 'cup', 'cup', 'ounce', 'tablespoon', 'pound', 'count', 'count', 'clove', 'cup', 'tablespoon', 'count', 'ounce']
['beef', 'onions', 'garlic', 'tomatoes', 'tomato sauce', 'chili powder', 'cumin', 'green bell pepper', 'kidney beans', 'bacon', 'onion', 'potatoes', 'eggs', 'cheddar', 'cottage cheese', 'pasta', 'olive oil', 'chicken breast', 'onion', 'yellow bell pepper', 'garlic', 'chicken broth', 'sherry', 'salt and pepper', 'cream cheese']


In [21]:
for i in range(len(unitList)):
    if unitList[i] in ["cup", "clove", "pound", "ounce", "clove", "stalk", "quart", "slice", "pint","gallon", "drop"]:
        unitList[i] = unitList[i].upper() + "S"
    if unitList[i] == "tablespoon":
        unitList[i] = "TBSP"
    if unitList[i] == "teaspoon":
        unitList[i] = "TSP"
    if unitList[i] == "leaf":
        unitList[i] = "LEAVES"
    if unitList[i] == "pinch":
        unitList[i] = "PINCHES"
    if unitList[i] == "dash":
        unitList[i] = "DE_DASH"

In [23]:
example = {"ingredients": []}

for ing, un, amt in zip(ingList, unitList,  amountList):
	newD = {"name": ing, 
					"quantityList":[{"unit": un, "amount": amt}],
					 "exclusiveOverride": "false"}
	example['ingredients'].append(newD)


# convert into JSON:
myJSON = json.dumps(example)

f = open('toAmazon.html','w')

html_content = """<html>
  <head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width">
    <title>replit</title>
    <link href="style.css" rel="stylesheet" type="text/css" />
  </head>
  <body>
		<form method="POST" action="https://www.amazon.com/afx/ingredients/landing">
     <input type="hidden" name="ingredients" value='""" + myJSON + """'>
     <input type="submit" value="Buy on Amazon" name = "button">
   </form>

    <script src="script.js"></script>
  </body>
</html>"""

f.write(html_content)
f.close()


In [29]:
driver = webdriver.Chrome()

driver.get("file://" + os.path.realpath("toAmazon.html"))

driver.find_element_by_name('button').click()

<ipython-input-29-a0d389db8a16>:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_name('button').click()
